# Data Science
# Exercise 4 - Collaborative Filtering
<br/>Student:
<br/>se21m024
<br/>Thomas Stummer
<br/><br/>The interpretation of the data can be found in the document <b><i>se21m024_Stummer_ex4_Coll_Filtering.pdf</i></b>.
<br/><br/>
Basic source code was taken from https://github.com/pratiknabriya/Recommender-System-Collaborative-Filtering-MovieLens and modified to fit the requirements of the assignment.
<br/><br/>
The data set was provided by<br>
F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets:
History and Context. ACM Transactions on Interactive Intelligent
Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages.
DOI=http://dx.doi.org/10.1145/2827872
<br/><br/>

# Import necessary dependencies

In [55]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
import numpy as np
import seaborn as sns
import math as math
import itertools as it
from sklearn.manifold import MDS
import sklearn.datasets as dt
from sklearn.metrics.pairwise import manhattan_distances, euclidean_distances
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice
import matplotlib.pyplot as plt
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice
from sklearn.preprocessing import OneHotEncoder

pd.options.display.max_columns = None

%matplotlib inline

## Import Data

In [56]:
# Source code taken from: https://jovian.ai/amitgold001/movielens-fastai



cols = ['userId','movieId', 'rating','timestamp']
ratings = pd.read_csv('./Data/ml-100k/u.data', delimiter='\t', header=None, names=cols)
ratings.sample(5)

# Source code taken from: https://github.com/pratiknabriya/Recommender-System-Collaborative-Filtering-MovieLens/blob/main/collaborative_filtering_movielense.ipynb

# Train-test split
from sklearn.model_selection import train_test_split

# Student ID: se21m024 -> random_state = 21024

X_train, X_test = train_test_split(ratings, test_size = 0.20, random_state = 24)

print(X_train.shape)
print(X_test.shape)

# pivot ratings into movie features
user_data = X_train.pivot(index = 'userId', columns = 'movieId', values = 'rating').fillna(0)
user_data.head()

# make a copy of train and test datasets
dummy_train = X_train.copy()
dummy_test = X_test.copy()

dummy_train['rating'] = dummy_train['rating'].apply(lambda x: 0 if x > 0 else 1)
dummy_test['rating'] = dummy_test['rating'].apply(lambda x: 1 if x > 0 else 0)

# The movies not rated by user is marked as 1 for prediction 
dummy_train = dummy_train.pivot(index = 'userId', columns = 'movieId', values = 'rating').fillna(1)

# The movies not rated by user is marked as 0 for evaluation 
dummy_test = dummy_test.pivot(index ='userId', columns = 'movieId', values = 'rating').fillna(0)

# dummy_train.head()
# dummy_test.head()

print(dummy_train)
print(dummy_test)


(80000, 4)
(20000, 4)
movieId  1     2     3     4     5     6     7     8     9     10    11    \
userId                                                                      
1         0.0   0.0   0.0   1.0   1.0   1.0   0.0   0.0   0.0   0.0   0.0   
2         0.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   0.0   1.0   
3         1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   
4         1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   0.0   
5         0.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
939       1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   0.0   1.0   1.0   
940       1.0   1.0   1.0   0.0   1.0   1.0   0.0   1.0   0.0   1.0   1.0   
941       0.0   1.0   1.0   1.0   1.0   1.0   0.0   1.0   1.0   1.0   1.0   
942       1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   
943       1.0   0.0   1.0   1.0   1.0   1.0   1.0   1.

## Algorithm 1: User-User Similarity matrix
Using Cosine similarity


In [57]:
# Source code taken from: https://github.com/pratiknabriya/Recommender-System-Collaborative-Filtering-MovieLens/blob/main/collaborative_filtering_movielense.ipynb

from sklearn.metrics.pairwise import cosine_similarity

# User Similarity Matrix using Cosine similarity as a similarity measure between Users
user_similarity = cosine_similarity(user_data)
user_similarity[np.isnan(user_similarity)] = 0
print(user_similarity)
print(user_similarity.shape)

# Predicting the User ratings on the movies
user_predicted_ratings = np.dot(user_similarity, user_data)
user_predicted_ratings

# np.multiply for cell-by-cell multiplication 

user_final_ratings = np.multiply(user_predicted_ratings, dummy_train)
user_final_ratings.head()


[[1.         0.12749131 0.04335795 ... 0.06502325 0.13515049 0.37295319]
 [0.12749131 1.         0.06767143 ... 0.14257081 0.16123985 0.12237756]
 [0.04335795 0.06767143 1.         ... 0.0841076  0.09209121 0.        ]
 ...
 [0.06502325 0.14257081 0.0841076  ... 1.         0.03385171 0.03615073]
 [0.13515049 0.16123985 0.09209121 ... 0.03385171 1.         0.1711502 ]
 [0.37295319 0.12237756 0.         ... 0.03615073 0.1711502  1.        ]]
(943, 943)


movieId        1          2          3           4          5          6     \
userId                                                                        
1          0.000000   0.000000   0.000000  194.097860  70.514738  16.172180   
2          0.000000  42.010575  43.549925   78.795848  31.690498  11.821509   
3        100.995218  20.508668  19.362439   34.759775  17.216300   4.888464   
4        111.438109  23.360723  19.908019   41.802880  17.347856   4.326971   
5          0.000000  94.897344  57.068788  154.025809  58.206840  10.275592   

movieId        7           8           9          10          11    \
userId                                                               
1          0.000000    0.000000    0.000000   0.000000    0.000000   
2        187.090062   99.261777  168.021162   0.000000  102.671045   
3         88.345892   43.731991   65.047181  18.812677   55.934257   
4         86.001993   48.874349   64.638018  15.135198    0.000000   
5        241.477865  174.702119  163.923323  50.463879  167.601316   

movieId        12          13          14          15         16         17    \
userId                                                                          
1          0.000000    0.000000    0.000000  210.739690   0.000000  79.926343   
2        144.272217    0.000000  106.317092  161.170224  20.845432  32.003291   
3         70.543476   37.026495   32.768036   62.573353   8.533228  19.887202   
4         72.946402   37.268620   33.819504   65.946028   6.940674  18.897664   
5        215.532234  103.538146   87.323492  167.561558  19.684723   0.000000   

movieId      18         19         20         21          22          23    \
userId                                                                       
1        0.000000   0.000000   0.000000   0.000000    0.000000    0.000000   
2        3.517419   0.000000  36.906758  28.516299  134.083195   78.427206   
3        1.580205  17.322698  14.198826  13.681603   65.899278   36.940006   
4        1.131593  15.812272  12.489804  14.473670   74.265697   38.325577   
5        4.835491  33.674893  30.239030   0.000000  237.984641  130.893072   

movieId       24          25         26         27          28         29    \
userId                                                                        
1         0.000000    0.000000   0.000000   0.000000    0.000000  75.820956   
2        76.927553  140.655462  29.635693  17.303758  111.806598  28.513249   
3        36.160840   53.469287  13.609993   9.484979   49.546507  15.184868   
4        37.459559   54.797017  13.699382  10.258025   56.195556  15.937644   
5         0.000000    0.000000  43.599753  34.848833  214.391278   0.000000   

movieId       30          31         32         33        34        35    \
userId                                                                     
1         0.000000  149.448244   0.000000   0.000000  0.000000  0.000000   
2        15.762612   61.817303  34.532971  37.214512  1.821809  1.701121   
3         5.872851   30.526410  16.423573  20.036482  1.201926  1.147945   
4         6.106403   33.903333  15.663147  21.429973  0.764313  1.378333   
5        27.771425  116.988455  57.859568  78.828092  3.445921  4.978865   

movieId      36        37         38         39         40         41    \
userId                                                                    
1        5.845235  3.809527   0.000000   0.000000   0.000000   0.000000   
2        2.890488  1.262635  37.419045  28.788808  16.758810  12.271836   
3        1.654018  0.825979  19.397447  14.604416   7.938171   6.748303   
4        1.451468  0.643685  20.798923  16.165216   7.739255   7.487836   
5        4.386443  3.367722  78.981894  56.475340   0.000000  27.126363   

movieId       42         43         44         45         46         47    \
userId                                                                      
1         0.000000   0.000000   0.000000   0.000000   0.000000   0.000000

### Top 5 movie recommendations for the User 42



In [58]:
user_final_ratings.iloc[42].sort_values(ascending = False)[0:5]

movieId
1      382.819959
56     346.559374
318    310.153532
168    306.110967
210    295.614854
Name: 43, dtype: float64

## Algorithm 2: Item-based collaborative filtering


In [59]:
movie_features = X_train.pivot(index = 'movieId', columns = 'userId', values = 'rating').fillna(0)
movie_features.head()


userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

## Item-Item Similarity matrix
Using Cosine similarity


In [60]:
from sklearn.metrics.pairwise import cosine_similarity

# Item Similarity Matrix using Cosine similarity as a similarity measure between Items
item_similarity = cosine_similarity(movie_features)
item_similarity[np.isnan(item_similarity)] = 0
print(item_similarity)
print("- "*10)
print(item_similarity.shape)

# Predicting the User ratings on the movies

item_predicted_ratings = np.dot(movie_features.T, item_similarity)
item_predicted_ratings


# Filtering the ratings only for the movies not already rated by the user for recommendation
# np.multiply for cell-by-cell multiplication 

item_final_ratings = np.multiply(item_predicted_ratings, dummy_train)
item_final_ratings.head()




[[1.         0.31697113 0.28767995 ... 0.         0.         0.05201389]
 [0.31697113 1.         0.18691343 ... 0.         0.08789381 0.        ]
 [0.28767995 0.18691343 1.         ... 0.         0.         0.10289915]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.08789381 0.         ... 0.         1.         0.        ]
 [0.05201389 0.         0.10289915 ... 0.         0.         1.        ]]
- - - - - - - - - - 
(1643, 1643)


movieId       1           2          3           4           5          6     \
userId                                                                         
1         0.000000    0.000000   0.000000  246.716873  152.817539  62.259473   
2         0.000000   31.008898  32.337738   39.798814   27.085758  16.540623   
3        19.322631   13.382176  13.184284   15.634446   12.248506   5.761955   
4        14.907988   10.113397   9.316696   12.287974    8.466436   3.973269   
5         0.000000  118.433222  78.581603  131.718756   84.575315  22.731378   

movieId        7           8          9          10          11          12    \
userId                                                                          
1          0.000000    0.000000   0.000000   0.000000    0.000000    0.000000   
2         50.232915   40.843374  49.213948   0.000000   43.071606   47.361342   
3         20.716287   15.630575  16.707699  11.517513   20.852231   20.935561   
4         14.268868   11.956146  11.747551   6.677790    0.000000   14.843779   
5        119.657756  119.883998  94.392940  61.295967  119.683990  123.718106   

movieId       13         14          15         16          17         18    \
userId                                                                        
1         0.000000   0.000000  180.356926   0.000000  163.334288   0.000000   
2         0.000000  38.739857   47.874517  24.595817   26.269351  11.299381   
3        15.435925  10.391790   15.695113   8.603261   14.663748   4.631932   
4        10.422646   7.622988   11.445923   4.826701    9.160910   2.354148   
5        88.905264  58.902876   93.015051  39.607698    0.000000  24.944686   

movieId       19         20         21          22         23         24    \
userId                                                                       
1         0.000000   0.000000   0.000000    0.000000   0.000000   0.000000   
2         0.000000  31.899636  26.744906   44.527243  35.151856  35.606622   
3         8.127559  11.164783  11.324379   19.218021  14.823464  15.737568   
4         5.453762   6.765061   8.240998   14.305041  10.804096  10.676743   
5        33.214975  43.965223   0.000000  135.613559  98.992635   0.000000   

movieId       25         26         27          28          29         30    \
userId                                                                        
1         0.000000   0.000000   0.000000    0.000000  186.956273   0.000000   
2        48.871019  28.684689  21.304517   41.701461   27.807366  18.716158   
3        15.925270  11.906111  10.821385   16.027104   13.246225   5.844547   
4        11.496541   8.021564   7.858547   12.096603    9.190926   4.129810   
5         0.000000  67.474928  73.296682  136.555532    0.000000  51.750432   

movieId        31         32          33         34         35         36    \
userId                                                                        
1        218.895385   0.000000    0.000000   0.000000   0.000000  69.899203   
2         36.328464  27.018935   28.484934   9.803898   5.637446  14.217608   
3         15.782384  11.675963   13.848021   6.558817   2.744222   7.622855   
4         11.517287   7.537007    9.869051   2.808213   2.331357   4.931937   
5        115.991450  73.171748  105.461840  30.839092  26.262989  37.129020   

movieId       37          38         39         40         41         42    \
userId                                                                       
1        51.764115    0.000000   0.000000   0.000000   0.000000   0.000000   
2         6.715238   30.968196  26.199656  21.530797  17.707096  32.564050   
3         4.161270   14.231165  11.707454   8.632675   9.111118  13.015377   
4         2.578153   10.267830   8.498025   5.797285   6.914930   9.388591   
5        31.712499  108.956626  86.635403   0.000000  72.797574   0.000000   

movieId       43         44         45         46          47         48    \
userId                               

### Top 5 movie recommendations for the User 42

In [61]:
item_final_ratings.iloc[42].sort_values(ascending = False)[0:5]

movieId
210    213.758875
423    212.735547
168    210.741268
56     208.795315
195    204.752782
Name: 43, dtype: float64

# Evaluation

Source code taken from: https://github.com/pratiknabriya/Recommender-System-Collaborative-Filtering-MovieLens/blob/main/collaborative_filtering_movielense.ipynb

Evaluation will we same as you have seen above for the prediction. The only difference being, you will evaluate for the movie already rated by the User instead of predicting it for the movie not rated by the user.


## Using User-User similarity

In [66]:
test_user_features = X_test.pivot(index = 'userId', columns = 'movieId', values = 'rating').fillna(0)
test_user_similarity = cosine_similarity(test_user_features)
test_user_similarity[np.isnan(test_user_similarity)] = 0

#print(test_user_similarity)
#print("- "*10)
#print(test_user_similarity.shape)

user_predicted_ratings_test = np.dot(test_user_similarity, test_user_features)
user_predicted_ratings_test

#print(user_predicted_ratings_test)
#print(dummy_test)

# Testing on the movies already rated by the user
test_user_final_rating = np.multiply(user_predicted_ratings_test, dummy_test)
#print(test_user_final_rating)


# We need to normalize the final rating values between range (0.5, 5)
from sklearn.preprocessing import MinMaxScaler

X = test_user_final_rating.copy() 

#X = X[X > 0] # only consider non-zero values as 0 means the user haven't rated the movies
#print(X)

scaler = MinMaxScaler(feature_range = (0.5, 5))
scaler.fit(X)
pred = scaler.transform(X)

#print(pred)

# total non-NaN value
total_non_nan = np.count_nonzero(~np.isnan(pred))
total_non_nan

test = X_test.pivot(index = 'userId', columns = 'movieId', values = 'rating').fillna(0)
#print(test)

# RMSE Score -> todo: change to MSE

from sklearn.metrics import mean_squared_error

mse = mean_squared_error(test, pred)
print(mse)



0.26029562467251444


## Using Item-Item Similarity

In [64]:


test_item_features = X_test.pivot(index = 'movieId', columns = 'userId', values = 'rating').fillna(0)
test_item_similarity = cosine_similarity(test_item_features)
test_item_similarity[np.isnan(test_item_similarity)] = 0 

print(test_item_similarity)
print("- "*10)
print(test_item_similarity.shape)

item_predicted_ratings_test = np.dot(test_item_features.T, test_item_similarity )
item_predicted_ratings_test

test_item_final_rating = np.multiply(item_predicted_ratings_test, dummy_test)
test_item_final_rating.head()

ratings['rating'].describe()

# But we need to normalize the final rating values between range (0.5, 5)

from sklearn.preprocessing import MinMaxScaler

X = test_item_final_rating.copy() 
X = X[X > 0] # only consider non-zero values as 0 means the user haven't rated the movies

scaler = MinMaxScaler(feature_range = (0.5, 5))
scaler.fit(X)
pred = scaler.transform(X)

print(pred)

# total non-NaN value
total_non_nan = np.count_nonzero(~np.isnan(pred))
total_non_nan

test = X_test.pivot(index = 'userId', columns = 'movieId', values = 'rating').fillna(0)
#test.head()


# MSE
mse = mean_squared_error(test, pred)
print(mse)






[[1.         0.03381301 0.0617448  ... 0.         0.08408278 0.        ]
 [0.03381301 1.         0.17608218 ... 0.         0.         0.        ]
 [0.0617448  0.17608218 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.08408278 0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
- - - - - - - - - - 
(1429, 1429)
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').